<a href="https://colab.research.google.com/github/aserkanturunc/Projects/blob/main/Brain_Cancer_Research_(GitHub).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1
!pip install lifelines
cancer_and_study_id={}
cancer_and_study_id['gbm'] = ['gbm_tcga_pub2013','gbm_tcga_pan_can_atlas_2018','gbm_tcga']

     |████████████████████████████████| 348 kB 28.8 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4049 sha256=4952efbff419322d04f46937069852a428e327fc8e08c628219676351379287a
  Stored in directory: /root/.cache/pip/wheels/9f/01/ee/1331593abb5725ff7d8c1333aee93a50a1c29d6ddda9665c9f
Successfully built autograd-gamma


In [ ]:
#2
import io
import os
import json
import requests
import hashlib
import pandas as pd
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter

In [ ]:
#3
from google.colab import files
uploaded = files.upload()#Excel Data.csv

# Dataset is now stored in a Pandas Dataframe

Saving EXCELdata.csv to EXCELdata.csv


In [ ]:
def getCbioData(url, method='GET', params= {}):
  cacheDir = "cachedData"
  if not os.path.exists(cacheDir):
    os.mkdir(cacheDir)
  fileName = "{0}/{1}_{2}".format(cacheDir, hashlib.md5(url.encode()).hexdigest(), method)
  if not os.path.exists(fileName):
    if method == 'GET':
      txt = requests.get(url).text
    else:
      txt = requests.post(url, json=params).text
    with open(fileName, 'w') as f:
      f.write(txt)
  else:
    with open(fileName, 'r') as f:
      txt = f.read()
  return json.loads(txt)




def get_all_MTgenes_and_IDs(genes): #returns a dictionary that has keys as gene id(number) and value as gene name
    MT_genes=genes
    MT_gene_and_ID={}#stores MT genes and their corresponding gene id
    all_genes = getCbioData('https://www.cbioportal.org/api/genes?direction=ASC&pageNumber=0&pageSize=10000000&projection=SUMMARY')
    # removed one redundant loop to speed up the code - MK
    for gene_info in all_genes:
      geneSymbol = gene_info['hugoGeneSymbol']
      if geneSymbol in MT_genes:
        MT_gene_and_ID[geneSymbol]=gene_info['entrezGeneId']
    return {value : key for (key, value) in MT_gene_and_ID.items()}
def create_df_patientid_and_gene_overexp(unique_patient_ids,patientGeneData):
    df=pd.DataFrame()
    df['PatientId']=[i for i in unique_patient_ids]
    for pid in unique_patient_ids:
      for gene in patientGeneData[pid].keys():
        df[gene+' overexpressed']=[patientGeneData[pid][gene][1] if gene in patientGeneData[pid].keys() else float('Nan') for pid in unique_patient_ids]
    return df
def create_df_patientid_and_gene_exp(unique_patient_ids,patientGeneData):
    df=pd.DataFrame()
    df['PatientId']=[i for i in unique_patient_ids]
    for pid in unique_patient_ids:
      for gene in patientGeneData[pid].keys():
        df[gene+' expression']=[patientGeneData[pid][gene][0]  if gene in patientGeneData[pid].keys() else float('Nan') for pid in unique_patient_ids]
    return df
def add_new_features_to_df_using_clinicaldata(df,unique_patient_ids,studies):
    patientClinicalValues = {}
    url = 'https://www.cbioportal.org/api/studies/{0}/clinical-data?clinicalDataType={1}&direction=ASC&pageNumber=0&pageSize=10000000&projection=SUMMARY'
    #Add ,'IDH1_MUTATION' to the attributes somehow, i think its not returned by api
    attributes = ['AGE', 'SEX', 'OS_MONTHS','OS_STATUS']
    for studyId in studies:
        response=requests.get(url.format(studyId,'PATIENT')).json()
        for row in response:
            attrName = row['clinicalAttributeId']
            if attrName not in attributes:
                continue
            patientId = row['patientId']
            if patientId not in patientClinicalValues.keys():
                patientClinicalValues[patientId] = {'AGE': None, 'SEX': None, 'OS_MONTHS': None, 'OS_STATUS':None}
            patientClinicalValues[patientId][attrName] = row['value']
    #unique_patient_ids_with_age_sex_os_months_status_cols=[]
    #for i in unique_patient_ids:
        #if i in patientClinicalValues.keys():
      
            #unique_patient_ids_with_age_sex_os_months_status_cols.append(i)
    df['OS_MONTHS']=[patientClinicalValues[patientId]['OS_MONTHS'] if patientId in patientClinicalValues else float('Nan')
                     for patientId in unique_patient_ids ]
    df['OS_STATUS']=[int(patientClinicalValues[patientId]['OS_STATUS'][0]) if patientId in patientClinicalValues else float('Nan')
                     for patientId in unique_patient_ids ]                 
    df['AGE']=[patientClinicalValues[patientId]['AGE'] if patientId in patientClinicalValues else float('Nan')
                     for patientId in unique_patient_ids ]
    df['SEX']=[patientClinicalValues[patientId]['SEX']=='Male' if patientId in patientClinicalValues else float('Nan')
                     for patientId in unique_patient_ids ]
      



In [ ]:

#Extracting the cbio data

#genes_we_want=serpin_genes #previously genes

geneIds = get_all_MTgenes_and_IDs(genes)

patientGeneData = {}
for studyId in cancer_and_study_id['gbm']:
  param = {
    "entrezGeneIds": list(geneIds.keys()),
    "sampleListId": "{0}_all".format(studyId)
  }
  #if gbm_tcga_pub: 
    #url=gbm_tcga_pub_mrna_median_all_sample_Zscores",
  url = 'https://www.cbioportal.org/api/molecular-profiles/{0}_rna_seq_v2_mrna_median_all_sample_Zscores/molecular-data/fetch'.format(studyId)
  data = getCbioData(url, 'POST', param)
  # data is a list of rows each of which is a dictionary that looks something like the below:
  # {'uniqueSampleKey': 'VENHQS0wMi0wMDAxLTAxOmdibV90Y2dhX3B1Yg', 'uniquePatientKey': 'VENHQS0wMi0wMDAxOmdibV90Y2dhX3B1Yg', 
  #     'entrezGeneId': 2, 'molecularProfileId': 'gbm_tcga_pub_mrna_median_all_sample_Zscores', 'sampleId': 'TCGA-02-0001-01', 
  #     'patientId': 'TCGA-02-0001', 'studyId': 'gbm_tcga_pub', 'value': 0.5089}
  for obj in data: 
    #try:    
    patientId = obj['patientId']
    gene = geneIds[obj['entrezGeneId']]#gene in this obj
    if patientId not in patientGeneData:
      patientGeneData[patientId] = {}
        #exp stores mrna expression of current gene in current patient Id
    exp=obj['value']
    overexp=0
    if exp>=1:
      overexp=1
    elif exp<=-1:
      overexp=-1  
    patientGeneData[patientId][gene]=(exp,overexp)
    #except:
      # just skip this row
      #pass
# show one sample patient data
patientId = list(patientGeneData.keys())[0]
print(patientId)
print(patientGeneData[patientId])
patientIds=patientGeneData.keys()
#df_CBIO_exp=create_df_patientid_and_gene_exp(patientIds,patientGeneData)
df_CBIO_overexp=create_df_patientid_and_gene_overexp(patientIds,patientGeneData)
#add_new_features_to_df_using_clinicaldata(df_CBIO_exp,patientIds,cancer_and_study_id['gbm'])
add_new_features_to_df_using_clinicaldata(df_CBIO_overexp,patientIds,cancer_and_study_id['gbm'])

TCGA-02-0047
{'A2M': (0.9457, 0), 'SERPINA3': (0.397, 0), 'ASIC2': (0.9296, 0), 'ASIC1': (0.4143, 0), 'ACTN1': (-0.2529, 0), 'ADAM8': (0.5905, 0), 'ADAM10': (0.9763, 0), 'AEBP1': (0.4222, 0), 'ACAN': (0.2805, 0), 'JAG1': (-0.6706, 0), 'AGT': (0.5966, 0), 'AKT1': (-0.1792, 0), 'AMBN': (-1.4268, -1), 'AMBP': (0.7978, 0), 'AMELX': (-0.8356, 0), 'AMELY': (-4.0918, -1), 'AMH': (0.2141, 0), 'ANGPT1': (0.7509, 0), 'ANGPT2': (0.5719, 0), 'ANXA1': (-0.7331, 0), 'ANXA2': (0.2168, 0), 'ANXA2P2': (0.285, 0), 'ANXA3': (-0.3803, 0), 'ANXA4': (-0.7746, 0), 'ANXA5': (-1.1683, -1), 'ANXA6': (1.1734, 1), 'ANXA7': (0.8194, 0), 'ANXA11': (1.1981, 1), 'ANXA13': (-1.2759, -1), 'FASLG': (-1.09, -1), 'AREG': (0.767, 0), 'RHOA': (0.1429, 0), 'SERPINC1': (0.0071, 0), 'ATOH1': (-1.4473, -1), 'AXL': (0.598, 0), 'BDNF': (-0.5285, 0), 'BGLAP': (-1.14, -1), 'BGN': (0.1406, 0), 'BMP1': (0.658, 0), 'BMP2': (0.3283, 0), 'BMP3': (-1.2596, -1), 'BMP4': (0.0893, 0), 'BMP5': (-0.1711, 0), 'BMP6': (0.1671, 0), 'BMP7': (0.21

In [ ]:
#Getting the excel data, does not have genes

df_excel = pd.read_csv(io.BytesIO(uploaded['EXCELdata.csv']))
df_excel.rename(columns={'Collection ID': 'PatientId'},inplace=True)
#df_excel.rename(columns={'Overall Survival Status': 'OS_STATUS'},inplace=True)
df_excel['WT YES']=(df_excel['IDH mutation status']=='WT').astype(int)
unique_pids=[]
indices_to_drop=[]
for index in df_excel.index:
    if df_excel.iloc[index]['PatientId'] in unique_pids:
        indices_to_drop.append(index)
    else:
        unique_pids.append(df_excel.iloc[index]['PatientId'])
indices_to_keep=[i for i in df_excel.index if i not in indices_to_drop]
df_excel_unique=df_excel.iloc[indices_to_keep]
for i in df_excel_unique['Sex'].index:
    if df_excel_unique['Sex'][i]=='Female':
        df_excel_unique['Sex'][i]=0
    elif df_excel_unique['Sex'][i]=='Male':
        df_excel_unique['Sex'][i]=1
df_excel_unique.rename(columns={'Collection ID': 'PatientId'},inplace=True)


#Merging DF EXCEL AND CBIO OVEREXP TO CREATE KPS OVEREXP


#KPS_exp=df_exp.merge(on='PatientId',right=df_excel_unique,how='inner')
#KPS_exp_all=df_exp.merge(on='PatientId',right=df_excel_unique,how='outer')
#KPS_exp.rename(columns={'OS_MONTHS_x': 'OS_MONTHS'},inplace=True)
#KPS_exp.drop(columns=['OS_MONTHS_y','Sex','Age at Diagnosis'],inplace=True)#dropping duplicate columns

df_KPS_overexp=df_CBIO_overexp.merge(on='PatientId',right=df_excel_unique,how='inner')
df_KPS_overexp.rename(columns={'OS_MONTHS_x': 'OS_MONTHS'},inplace=True)
#KPS_overexp_all=df_exp.merge(on='PatientId',right=df_excel_unique,how='outer')
df_KPS_overexp.drop(columns=['OS_MONTHS_y','Sex','Age at Diagnosis','Overall Survival Status'],inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/c

In [ ]:
#CLEANING THE NON EXCEL DF FOR COX
#df_exp_na_row_dropped=df_exp.drop(index=155) #index 155 has NAN on osmonths, sex, and age
df_CBIO_overexp_na_row_dropped=df_CBIO_overexp.drop(index=155)
#df_exp_clean=df_exp_na_row_dropped.dropna(axis=1) #dropping columns(genes) that do not have have value in every row(patient)
df_CBIO_overexp_clean=df_CBIO_overexp_na_row_dropped.dropna(axis=1)
#clean=no NAN

#CLEANING THE EXCEL DF FOR COX
#KPS_exp_clean=KPS_exp.drop(columns=['IDH mutation status'])
#df_KPS_overexp_clean=df_KPS_overexp.drop(columns=['IDH mutation status','KPS SCORE','Grade (LGG or HGG)'])
#KPS_exp_clean=KPS_exp.dropna(axis=0)
#df_KPS_overexp_clean=df_KPS_overexp_clean.dropna(axis=0)

In [ ]:
length=len(list(df_KPS_overexp_clean['WT YES']))
length



69

In [ ]:
from sklearn.feature_selection import VarianceThreshold

var_thr = VarianceThreshold(threshold = 0.1) #Removing both constant and quasi-constant
PatientIds=df_KPS_overexp_clean['PatientId']
OS_STATUS=df_KPS_overexp_clean['OS_STATUS']
var_thr.fit(df_KPS_overexp_clean.drop(columns=['PatientId','OS_STATUS']))
indices=var_thr.get_support(indices=True)
indices+=1
df_KPS_overexp_clean=df_KPS_overexp_clean.iloc[:,indices]
df_KPS_overexp_clean['PatientId']=PatientIds
df_KPS_overexp_clean['OS_STATUS']=OS_STATUS
df_KPS_overexp_clean['WT YES']=df_KPS_overexp['WT YES']
df_KPS_overexp_clean

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,A2M overexpressed,SERPINA3 overexpressed,ASIC2 overexpressed,ASIC1 overexpressed,ACTN1 overexpressed,ADAM8 overexpressed,ADAM10 overexpressed,AEBP1 overexpressed,ACAN overexpressed,JAG1 overexpressed,AGT overexpressed,AKT1 overexpressed,AMBN overexpressed,AMBP overexpressed,AMH overexpressed,ANGPT1 overexpressed,ANGPT2 overexpressed,ANXA1 overexpressed,ANXA2 overexpressed,ANXA2P2 overexpressed,ANXA3 overexpressed,ANXA4 overexpressed,ANXA5 overexpressed,ANXA6 overexpressed,ANXA7 overexpressed,ANXA11 overexpressed,ANXA13 overexpressed,FASLG overexpressed,AREG overexpressed,RHOA overexpressed,SERPINC1 overexpressed,ATOH1 overexpressed,AXL overexpressed,BDNF overexpressed,BGLAP overexpressed,BGN overexpressed,BMP1 overexpressed,BMP2 overexpressed,BMP3 overexpressed,BMP4 overexpressed,...,AGRN overexpressed,CLEC12B overexpressed,C1QTNF9B overexpressed,CCL4L1 overexpressed,CLEC17A overexpressed,HRNR overexpressed,FLG2 overexpressed,SFTA2 overexpressed,LIN28B overexpressed,C1QL3 overexpressed,C1QTNF8 overexpressed,GDF6 overexpressed,ELFN1 overexpressed,MUC21 overexpressed,VWC2L overexpressed,HAPLN4 overexpressed,TNFSF12-TNFSF13 overexpressed,CCL3L3 overexpressed,ASTL overexpressed,TMEM150C overexpressed,CLEC18B overexpressed,CBLN3 overexpressed,COL6A4P2 overexpressed,SERPINE3 overexpressed,ANXA8 overexpressed,CFC1B overexpressed,SFTPA1 overexpressed,LGALS9C overexpressed,INS-IGF2 overexpressed,MUC5B overexpressed,CLEC19A overexpressed,SFTPA2 overexpressed,HOTAIR overexpressed,GDF2 overexpressed,OS_MONTHS,OS_STATUS,AGE,SEX,PatientId,WT YES
0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,-1,1,0,1,-1,-1,0,0,0,-1,0,0,-1,0,0,0,-1,0,...,0,0,0,0,0,0,0,-1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,-1,0,0,-1,-1,-1,-1,-1,0,0,0,-1,14.72,1.0,78,True,TCGA-02-0047,1
1,0,1,0,0,1,1,1,1,1,0,0,0,-1,-1,0,-1,0,1,0,1,-1,1,0,0,0,0,0,0,0,0,0,-1,0,0,-1,1,0,1,0,0,...,0,-1,-1,0,0,0,0,-1,-1,1,0,1,0,-1,0,1,0,0,-1,0,1,0,0,0,-1,0,0,0,0,0,-1,0,-1,0,47.57,1.0,63,False,TCGA-06-0125,1
2,0,-1,-1,1,-1,0,0,-1,0,0,-1,0,-1,0,1,-1,-1,-1,-1,-1,0,0,-1,0,0,0,-1,0,-1,-1,0,0,1,1,0,-1,0,1,0,1,...,0,0,0,1,0,0,-1,-1,1,0,1,1,0,0,-1,0,-1,1,-1,-1,0,-1,1,1,0,-1,-1,0,0,0,0,-1,0,-1,33.64,1.0,30,True,TCGA-06-0129,0
3,0,0,-1,-1,0,0,1,0,-1,1,-1,0,-1,0,0,0,-1,0,1,1,0,1,0,0,0,1,0,1,0,0,-1,-1,1,0,-1,1,1,0,0,0,...,0,1,0,1,0,0,-1,-1,-1,0,-1,1,0,-1,-1,-1,1,1,0,0,0,1,0,-1,0,-1,-1,0,-1,0,-1,-1,-1,-1,12.94,1.0,54,True,TCGA-06-0130,1
4,0,0,1,0,0,0,0,0,0,-1,0,0,-1,-1,-1,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,-1,0,0,0,0,-1,0,0,0,...,-1,0,-1,-1,1,0,-1,0,0,1,-1,0,0,-1,0,0,1,-1,0,0,-1,0,0,-1,0,1,-1,1,1,0,-1,1,0,-1,25.33,1.0,49,True,TCGA-06-0132,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0,1,1,-1,1,1,0,0,0,0,-1,0,-1,0,0,1,0,0,1,1,1,0,0,1,1,1,0,0,0,1,-1,-1,0,0,0,1,1,0,0,0,...,0,-1,0,0,-1,1,0,-1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,-1,0,1,-1,-1,-1,0,-1,0,0,-1,7.03,0.0,36,True,TCGA-19-4065,1
66,0,1,0,0,0,0,-1,0,0,-1,0,-1,0,1,-1,0,-1,0,0,0,1,1,0,0,1,1,1,1,0,1,-1,-1,0,0,1,0,-1,0,0,0,...,-1,0,-1,0,1,0,-1,-1,0,1,0,-1,-1,-1,0,0,1,0,-1,1,0,1,-1,-1,1,0,0,1,0,-1,1,1,0,-1,15.11,1.0,49,True,TCGA-14-0736,0
67,-1,0,0,-1,0,0,0,0,0,0,0,0,-1,-1,0,1,-1,1,0,1,1,1,1,0,1,0,0,0,0,0,-1,-1,0,1,1,-1,-1,0,0,0,...,-1,0,0,0,0,-1,-1,-1,-1,0,0,-1,-1,-1,-1,0,0,0,-1,0,1,0,-1,0,1,0,-1,0,-1,0,-1,0,0,-1,32.03,1.0,58,False,TCGA-14-1402,0
68,0,1,0,0,0,0,0,0,1,-1,0,0,0,-1,0,0,0,0,0,0,0,0,-1,0,0,0,1,0,0,0,1,-1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1,0,0,-1,-1,1,-1,1,0,-1,0,0,-1,0,0,0,0,0,-1,-1,21.88,1.0,48,False,TCGA-19-0957,0


In [ ]:

#COX ANALYSIS

#MULTIVARIATE

#1) Multivariate analysis 160 PATIENTS, ALL GENES overexpressed vals (1,0,-1), PLUS OS MONTHS, AGE, SEX 

#2) Multivariate analysis 160 PATIENTS, ALL GENES expression (z score), PLUS OS MONTHS, AGE, SEX 

#3) Multivariate analysis 53 PATIENTS, ALL GENES overexpressed vals (1,0,-1), PLUS OS MONTHS, AGE, SEX, KPS, IDH, MGMT, LGG/HGG

#4) Multivariate analysis 53 PATIENTS, ALL GENES expression (z score) PLUS OS MONTHS, AGE, SEX, KPS, IDH, MGMT, LGG/HGG

#UNIVARIATE

#5) Univariate analysis (for gene in genes: (gene+os months,age,sex) 160 PATIENTS, ALL GENES overexpressed vals (1,0,-1), PLUS OS MONTHS, AGE, SEX 

#6) Univariate analysis (for gene in genes: (gene+os months,age,sex) 160 PATIENTS, ALL GENES expression (z score), PLUS OS MONTHS, AGE, SEX 

#7) Univariate analysis (for gene in genes: (gene+os months,age,sex,KPS,IDH,LGG/HGG) 53 PATIENTS, ALL GENES overexpressed vals (1,0,-1), PLUS OS MONTHS, AGE, SEX, KPS, IDH, MGMT, LGG/HGG

#8) Univariate analysis (for gene in genes: (gene+os months,age,sex,KPS,IDH,LGG/HGG) 53 PATIENTS, ALL GENES expression (z score) PLUS OS MONTHS, AGE, SEX, KPS, IDH, MGMT, LGG/HGG

#start 1

#df datatable contains 160 patients, all gene overexpressed and all gene expressed

#df_overexpressed= will contain only overexpressed
#df_expresssion= will contain only expression
#1-4

def cox(data): #df with columns selected 
    cph=CoxPHFitter(penalizer=0.1)
    cph.fit(data,'OS_MONTHS','OS_STATUS')
    return cph.summary

def multivariate(df, WT=False,onlyP_and_HR=True):
  if onlyP_and_HR:
    return cox(df.drop(columns=['PatientId']))[['p','exp(coef)']] #multivariate 
  else:
    return cox(df.drop(columns=['PatientId'])) #multivariate 
  

def univariate(df,WT=False): 
    uni_gene_and_cox={}
    if WT:
      for gene in df.drop(columns=['PatientId','SEX','AGE','OS_MONTHS','OS_STATUS','WT YES']).columns: #apply this for gene exp and gene oveexp
      #ADD OTHER FEATURES LIKE WT, MGMT
        uni_gene_and_cox[gene]=cox(df[[gene,'SEX','AGE','OS_MONTHS','OS_STATUS','WT YES']])
    else:
      for gene in df.drop(columns=['PatientId','SEX','AGE','OS_MONTHS','OS_STATUS']).columns: #apply this for gene exp and gene oveexp
        uni_gene_and_cox[gene]=cox(df[[gene,'SEX','AGE','OS_MONTHS','OS_STATUS']])
    return uni_gene_and_cox
#dataframes:
# df_exp, df_overexp
# KPS_exp, KPS_overexp

#MULTIVARIATE
#return a df that stores cox results for multivariate analysis on the specified df
#1
#df_CBIO_exp_mv=multivariate(df_exp_clean)  #DONE
#2
#df_CBIO_overexp_mv=multivariate(df_overexp_clean) #DONE
#3
#df_KPS_exp_mv=multivariate(KPS_exp, KPS=True) #DONE
#4
#df_KPS_overexp_mv=multivariate(KPS_overexp, KPS=True) #NOT COMPLETE

#UNIVARIATE
#return dictionary that stores cox results df for every gene
#5
#dict_CBIO_exp_uv=univariate(df_exp_clean) #DONE
#6
dict_CBIO_overexp_uv=univariate(df_CBIO_overexp_clean) #DONE
#7
#dict_KPS_exp_uv=univariate(KPS_exp, KPS=True) #DONE
#8
dict_KPS_overexp_uv=univariate(df_KPS_overexp_clean, WT=True) #NOT COMPLETE



In [ ]:
#TO DO 

#1
#SELECT THE TOP 20 GENES WITH HIGHEST EXP (COEF) IN UNIVARIATE (FROM THE ANALYSIS #6 AND #8)  
#2
#AND APPLY MULTIVARIATE TO THEM
#3
# Compare kps vs non kps genes
#4 
#present in human readable format

In [ ]:
cox(df_KPS_overexp_clean[['WT YES','OS_STATUS','OS_MONTHS']])

,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
covariate,,,,,,,,,,
WT YES,0.07293,1.075655,0.315887,-0.546196,0.692056,0.579148,1.99782,0.230874,0.817413,0.290863


In [ ]:
#Univariates for OVEREXPRESSION value KPS

gene_and_HR_KPS={}
for gene in dict_KPS_overexp_uv:
  cph_summary_df_KPS=dict_KPS_overexp_uv[gene]
  gene_and_HR_KPS[gene]=cph_summary_df_KPS['exp(coef)'][gene]
gene_and_HR_KPS #THIS stores each gene and its univariate HR 

sorted_gene_and_HR_KPS={k:v for k,v in sorted(gene_and_HR_KPS.items(), key=lambda item: item[1],reverse=True)}
sorted_gene_Df_KPS=pd.DataFrame({'gene':sorted_gene_and_HR_KPS.keys(),'exp':sorted_gene_and_HR_KPS.values()}) #this df stores each gene and its uv HR, sorted

n=100#TOP N GENES
TOP_UV_GENES_KPS=list(sorted_gene_Df_KPS['gene'][:n])
print(TOP_UV_GENES_KPS)


['C1QTNF7 overexpressed', 'SERPINA10 overexpressed', 'IFNA21 overexpressed', 'DHH overexpressed', 'THBS4 overexpressed', 'EPGN overexpressed', 'IFNK overexpressed', 'IFNA13 overexpressed', 'RPTN overexpressed', 'COL17A1 overexpressed', 'NYX overexpressed', 'CILP overexpressed', 'SFRP4 overexpressed', 'SVEP1 overexpressed', 'WNT5B overexpressed', 'KAZALD1 overexpressed', 'WNT4 overexpressed', 'FGF10 overexpressed', 'SPOCK1 overexpressed', 'MMP24 overexpressed', 'CLEC19A overexpressed', 'IFNB1 overexpressed', 'FMOD overexpressed', 'IFNA1 overexpressed', 'CBLN3 overexpressed', 'COMP overexpressed', 'FGFBP1 overexpressed', 'HTRA1 overexpressed', 'JAK2 overexpressed', 'PODNL1 overexpressed', 'ITLN1 overexpressed', 'CCBE1 overexpressed', 'KCNK2 overexpressed', 'CXCL9 overexpressed', 'CLEC10A overexpressed', 'TLL2 overexpressed', 'FBLN5 overexpressed', 'FREM2 overexpressed', 'IL25 overexpressed', 'AKT1 overexpressed', 'ADAMTS7 overexpressed', 'SERPINB5 overexpressed', 'LEFTY1 overexpressed', 

In [ ]:
sorted_gene_Df_KPS[:50]

,gene,exp
0,C1QTNF7 overexpressed,2.471621
1,SERPINA10 overexpressed,2.193159
2,IFNA21 overexpressed,2.179469
3,DHH overexpressed,2.114849
4,THBS4 overexpressed,2.107920
5,EPGN overexpressed,2.028335
6,IFNK overexpressed,1.988278
7,IFNA13 overexpressed,1.865284
8,RPTN overexpressed,1.862330
9,COL17A1 overexpressed,1.802644


In [ ]:

#for gene in sorted_gene_Df_KPS['gene'][:n]:
 # print(dict_KPS_overexp_uv[gene][['exp(coef)','p']])



#Univariates for OVEREXPRESSION value for without KPS


#for gene in TOP_UV_GENES_CBIO:#STAR
  #print(dict_CBIO_overexp_uv[gene][['exp(coef)','p']])



#APPLYING MULTIVARIATE TO TOP 20 GENES IN KPS AND CBIO


TOP_UV_GENES_CBIO_mv=multivariate(df_CBIO_overexp_clean[TOP_UV_GENES_CBIO+['PatientId','OS_MONTHS','OS_STATUS']], KPS=False)
TOP_UV_GENES_KPS_mv=multivariate(df_KPS_overexp_clean[TOP_UV_GENES_KPS+['PatientId','OS_MONTHS','OS_STATUS']], KPS=True)

#Adding how many over, normal, under patients 

over=[]
normal=[]
under=[]
for gene in TOP_UV_GENES_CBIO_mv.index:
  over.append(sum(df_CBIO_overexp_clean[gene]==1))
  normal.append(sum(df_CBIO_overexp_clean[gene]==0))
  under.append(sum(df_CBIO_overexp_clean[gene]==-1))

TOP_UV_GENES_CBIO_mv['# overexp patients']=over
TOP_UV_GENES_CBIO_mv['# normalexp patients']=normal
TOP_UV_GENES_CBIO_mv['# underexp patients']=under
TOP_UV_GENES_CBIO_mv['# total patients']=[160]*n
TOP_UV_GENES_CBIO_mv.sort_values(by='exp(coef)',inplace=True,ascending=False)

over=[]
normal=[]
under=[]
for gene in TOP_UV_GENES_KPS_mv.index:
  over.append(sum(df_KPS_overexp_clean[gene]==1))
  normal.append(sum(df_KPS_overexp_clean[gene]==0))
  under.append(sum(df_KPS_overexp_clean[gene]==-1))

TOP_UV_GENES_KPS_mv['# overexp patients']=over
TOP_UV_GENES_KPS_mv['# normalexp patients']=normal
TOP_UV_GENES_KPS_mv['# underexp patients']=under
TOP_UV_GENES_KPS_mv['# total patients']=[57]*n
TOP_UV_GENES_KPS_mv.sort_values(by='exp(coef)',inplace=True,ascending=False)


NameError: ignored

In [ ]:
gene_and_HR_CBIO={}

for gene in dict_CBIO_overexp_uv:
  cph_summary_df_CBIO=dict_CBIO_overexp_uv[gene]
  gene_and_HR_CBIO[gene]=cph_summary_df_CBIO['exp(coef)'][gene]
gene_and_HR_CBIO #THIS stores each gene and its univariate HR 

sorted_gene_and_HR_CBIO={k:v for k,v in sorted(gene_and_HR_CBIO.items(), key=lambda item: item[1],reverse=True)}
sorted_gene_Df_CBIO=pd.DataFrame({'gene':sorted_gene_and_HR_CBIO.keys(),'exp':sorted_gene_and_HR_CBIO.values()}) #this df stores each gene and its uv HR, sorted

n=100# GETTING TOP N GENES FROM UNIVARIATE ANALYSIS
TOP_UV_GENES_CBIO=list(sorted_gene_Df_CBIO['gene'][:n])
print(TOP_UV_GENES_CBIO)
sorted_gene_Df_CBIO[:50]

In [ ]:
TOP_UV_GENES_WT_mv=multivariate(df_KPS_overexp_clean[TOP_UV_GENES_CBIO+['PatientId','OS_MONTHS','OS_STATUS']], KPS=False)

In [ ]:
pd.set_option('display.max_rows', 1000)
TOP_UV_GENES_CBIO_mv


NameError: ignored

In [ ]:
COMMON_GENES=TOP_UV_GENES_KPS_mv.merge(right=TOP_UV_GENES_CBIO_mv,on='covariate',how='inner',suffixes=('_KPS','_CBIO'))
gene_names=[]
for i in list(COMMON_GENES.index):
  gene_names.append(i.split(' ')[0])
gene_names

['ANXA10',
 'ADAMDEC1',
 'THBS4',
 'SERPINA10',
 'CILP',
 'FGF10',
 'SERPINI1',
 'IFNA1',
 'CLEC19A',
 'RPTN',
 'CPN2',
 'MYL2',
 'IFNK',
 'HTRA1',
 'CCBE1',
 'EPGN',
 'FBLN5',
 'IFNA13',
 'ITLN1',
 'FGF5',
 'ADAM19',
 'FGFBP2',
 'INHBA',
 'CTSD',
 'SERPINB5',
 'REG4',
 'MMP23B',
 'TLL2',
 'GPC4',
 'SERPINB13',
 'IL19',
 'WNT4',
 'ADAMTSL1']

In [ ]:
COMMON_GENES

,p_KPS,exp(coef)_KPS,# overexp patients_KPS,# normalexp patients_KPS,# underexp patients_KPS,# total patients_KPS,p_CBIO,exp(coef)_CBIO,# overexp patients_CBIO,# normalexp patients_CBIO,# underexp patients_CBIO,# total patients_CBIO
covariate,,,,,,,,,,,,
ANXA10 overexpressed,0.021544,3.357782,2,9,46,57,0.252717,0.756622,5,25,130,160
ADAMDEC1 overexpressed,0.036905,2.803476,11,34,12,57,0.156620,1.340325,29,100,31,160
THBS4 overexpressed,0.123526,2.444334,7,43,7,57,0.589839,1.132218,31,107,22,160
SERPINA10 overexpressed,0.315127,1.879072,2,6,49,57,0.608102,0.831416,2,18,140,160
CILP overexpressed,0.352350,1.710863,6,45,6,57,0.247777,1.397317,16,126,18,160
FGF10 overexpressed,0.340314,1.616312,3,15,39,57,0.028770,1.598195,12,38,110,160
SERPINI1 overexpressed,0.439765,1.553184,7,42,8,57,0.095526,1.590082,20,116,24,160
IFNA1 overexpressed,0.606901,1.509061,1,4,52,57,0.142279,1.705291,3,12,145,160
CLEC19A overexpressed,0.528922,1.415517,2,10,45,57,0.196603,1.362607,4,32,124,160


In [ ]:
#Multivariates of all genes
#Multivariates
#Multivariates

n=20
def sort_by_expcoef(df,n):
  return df.sort_values(by='exp(coef)',ascending=False).iloc[:n][['exp(coef)','p']]

sorted_df_exp_mv=sort_by_expcoef(df_exp_mv,n)
sorted_df_overexp_mv=sort_by_expcoef(df_overexp_mv,n)
sorted_df_KPS_exp_mv=sort_by_expcoef(df_KPS_exp_mv,n)
sorted_df_KPS_overexp_mv=sort_by_expcoef(df_KPS_overexp_mv,n)
print('EXPRESSION Z SCORE MULTIVARIATE\n ',sorted_df_exp_mv)
print('\nOVEREXPRESSION VALUES (1,0,-1) MULTIVARIATE\n ',sorted_df_overexp_mv)
print('\nEXPRESSION Z SCORES WITH KPS DATA MULTIVARIATE\n ',sorted_df_KPS_exp_mv)
print('\OVEREXPRESSION VALUES (1,0,-1) WITH KPS DATA MULTIVARIATE\n ',sorted_df_KPS_overexp_mv)


In [ ]:
serpin_genes=[]
for gene in genes:
  if gene[:6]=='SERPIN':
    serpin_genes.append(gene)


In [ ]:
genes="""
ABI3BP
ADIPOQ
AEBP1
AGRN
AMBN
AMELX
AMELY
BGLAP
BMPER
BSPH1
CDCP2
CILP
CILP2
COCH
COLQ
COMP
CRELD1
CRELD2
CRIM1
CRISPLD1
CRISPLD2
CTGF
CTHRC1
CYR61
DDX26B
DMBT1
DMP1
DPT
DSPP
ECM1
ECM2
EDIL3
EFEMP1
EFEMP2
EGFLAM
ELN
ELSPBP1
EMID1
EMILIN1
EMILIN2
EMILIN3
EYS
FBLN1
FBLN2
FBLN5
FBLN7
FBN1
FBN2
FBN3
FGA
FGB
FGG
FGL1
FGL2
FN1
FNDC1
FNDC7
FNDC8
FRAS1
GAS6
GLDN
HMCN1
HMCN2
IBSP
IGFALS
IGFBP1
IGFBP2
IGFBP3
IGFBP4
IGFBP5
IGFBP6
IGFBP7
IGFBPL1
IGSF10
KAL1
KCP
LAMA1
LAMA2
LAMA3
LAMA4
LAMA5
LAMB1
LAMB2
LAMB3
LAMB4
LAMC1
LAMC2
LAMC3
LGI1
LGI2
LGI3
LGI4
LRG1
LTBP1
LTBP2
LTBP3
LTBP4
MATN1
MATN2
MATN3
MATN4
MEPE
MFAP1
MFAP2
MFAP3
MFAP4
MFAP5
MFGE8
MGP
MMRN1
MMRN2
MXRA5
NDNF
NELL1
NELL2
NID1
NID2
NOV
NPNT
NTN1
NTN3
NTN4
NTN5
NTNG1
NTNG2
OIT3
OTOG
OTOL1
PAPLN
PCOLCE
PCOLCE2
POMZP3
POSTN
PXDN
PXDNL
RELN
RSPO1
RSPO2
RSPO3
RSPO4
SBSPON
SLIT1
SLIT2
SLIT3
SMOC1
SMOC2
SNED1
SPARC
SPARCL1
SPON1
SPON2
SPP1
SRPX
SRPX2
SSPO
SVEP1
TECTA
TECTB
TGFBI
THBS1
THBS2
THBS3
THBS4
THSD4
TINAG
TINAGL1
TNC
TNFAIP6
TNN
TNR
TNXB
TSKU
TSPEAR
VIT
VTN
VWA1
VWA2
VWA3A
VWA3B
VWA5A
VWA5B1
VWA5B2
VWA7
VWA9

VWCE
VWDE
VWF
WISP1
WISP2
WISP3
ZP1
ZP2
ZP3
ZP4
ZPLD1
COL10A1
COL11A1
COL11A2
COL12A1
COL13A1
COL14A1
COL15A1
COL16A1
COL17A1
COL18A1
COL19A1
COL1A1
COL1A2
COL20A1
COL21A1
COL22A1
COL23A1
COL24A1
COL25A1
COL26A1
COL27A1
COL28A1
COL2A1
COL3A1
COL4A1
COL4A2
COL4A3
COL4A4
COL4A5
COL4A6
COL5A1
COL5A2
COL5A3
COL6A1
COL6A2
COL6A3
COL6A5
COL6A6
COL7A1
COL8A1
COL8A2
COL9A1
COL9A2
COL9A3
ACAN
ASPN
BCAN
BGN
CHAD
CHADL
DCN
EPYC
ESM1
FMOD
HAPLN1
HAPLN2
HAPLN3
HAPLN4
HSPG2
IMPG1
IMPG2
KERA
LUM
NCAN
NYX
OGN
OMD
OPTC
PODN
PODNL1
PRELP
PRG2
PRG3
PRG4
SPOCK1
SPOCK2
SPOCK3
SRGN
VCAN
ANXA1
ANXA10
ANXA11
ANXA13
ANXA2
ANXA3
ANXA4
ANXA5
ANXA6
ANXA7
ANXA8
ANXA8L1
ANXA9
C1QA
C1QB
C1QC
C1QL1
C1QL2
C1QL3
C1QL4
C1QTNF1
C1QTNF2
C1QTNF3
C1QTNF4
C1QTNF5
C1QTNF6
C1QTNF7
C1QTNF8
C1QTNF9
CD209
CLC
CLEC10A
CLEC11A
CLEC12A
CLEC12B
CLEC14A
CLEC17A
CLEC18A
CLEC18B
CLEC18C
CLEC19A
CLEC1A
CLEC1B
CLEC2A
CLEC2B
CLEC2D
CLEC2L
CLEC3A
CLEC3B
CLEC4A
CLEC4C
CLEC4D
CLEC4E
CLEC4F
CLEC4G
CLEC4M
CLEC5A
CLEC6A
CLEC7A
CLEC9A
COLEC10
COLEC11
COLEC12
CSPG4
CSPG5
ELFN1
ELFN2
EMCN
FCN1
FCN2
FCN3
FREM1
FREM2
FREM3
GPC1
GPC2
GPC3
GPC4
GPC5
GPC6
GREM1
GRIFIN
HPX
HSPC159
ITLN1
ITLN2
LGALS1
LGALS12
LGALS13
LGALS14
LGALS16
LGALS2
LGALS3
LGALS4
LGALS7
LGALS8
LGALS9
LGALS9B
LGALS9C
LMAN1
LMAN1L
MBL2
MUC1
MUC12
MUC13
MUC15
MUC16
MUC17
MUC19
MUC2
MUC20
MUC21
MUC22
MUC3A
MUC4
MUC5AC
MUC5B
MUC6
MUC7
MUC8
MUCL1
OVGP1
PARM1
PLXDC1
PLXDC2
PLXNA1
PLXNA2
PLXNA3
PLXNA4
PLXNB1
PLXNB2
PLXNB3
PLXNC1
PLXND1
PROL1
REG1A
REG1B
REG3A
REG3G
REG4
SDC1
SDC2
SDC3
SDC4
SEMA3A
SEMA3B
SEMA3C
SEMA3D
SEMA3E
SEMA3F
SEMA3G
SEMA4A
SEMA4B
SEMA4C
SEMA4D
SEMA4F
SEMA4G
SEMA5A
SEMA5B
SEMA6A
SEMA6B
SEMA6C
SEMA6D
SEMA7A
SFTA2
SFTA3
SFTPA1
SFTPA2
SFTPB
SFTPC
SFTPD
A2M
A2ML1
ADAM10
ADAM11
ADAM12
ADAM15
ADAM17
ADAM18
ADAM19
ADAM2
ADAM20
ADAM21
ADAM22
ADAM23
ADAM28
ADAM29
ADAM30
ADAM32
ADAM33
ADAM7
ADAM8
ADAM9
ADAMDEC1
ADAMTS1
ADAMTS10
ADAMTS12
ADAMTS13
ADAMTS14
ADAMTS15
ADAMTS16
ADAMTS17
ADAMTS18
ADAMTS19
ADAMTS2
ADAMTS20
ADAMTS3
ADAMTS4
ADAMTS5
ADAMTS6
ADAMTS7
ADAMTS8
ADAMTS9
ADAMTSL1
ADAMTSL2
ADAMTSL3
ADAMTSL4
ADAMTSL5
AGT
AMBP
ASTL
BMP1
C17orf58
CD109
CELA1
CELA2A
CELA2B
CELA3A
CELA3B
CPAMD8
CPN2
CST1
CST11
CST2
CST3
CST4
CST5
CST6
CST7
CST8
CST9
CST9L
CSTA
CSTB
CSTL1
CTSA
CTSB
CTSC
CTSD
CTSE
CTSF
CTSG
CTSH
CTSK
CTSL
CTSO
CTSS
CTSV
CTSW
CTSZ
EGLN1
EGLN2
EGLN3
ELANE
F10
F12
F13A1
F13B
F2
F7
F9
FAM20A
FAM20B
FAM20C
HABP2
HMSD
HPSE
HPSE2
HRG
HTRA1
HTRA3
HTRA4
HYAL1
HYAL2
HYAL3
HYAL4
ITIH1
ITIH2
ITIH3
ITIH4
ITIH5
ITIH6
KAZALD1
KNG1
KY
LEPRE1
LEPREL1
LEPREL2
LOX
LOXL1
LOXL2
LOXL3
LOXL4
LPA
MASP1
MASP2
MEP1A
MEP1B
MMP1
MMP10
MMP11
MMP12
MMP13
MMP14
MMP15
MMP16
MMP17
MMP19
MMP2
MMP20
MMP21
MMP23B
MMP24
MMP25
MMP26
MMP27
MMP28
MMP3
MMP7
MMP8
MMP9
NGLY1
OGFOD1
OGFOD2
P4HA1
P4HA2
P4HA3
P4HTM
PAMR1
PAPPA
PAPPA2
PCSK5
PCSK6
PI3
PLAT
PLAU
PLG
PLOD1
PLOD2
PLOD3
PRSS1
PRSS12
PRSS2
PRSS3
PZP
SERPINA1
SERPINA10
SERPINA11
SERPINA12
SERPINA2
SERPINA3
SERPINA4
SERPINA5
SERPINA6
SERPINA7
SERPINA9
SERPINB1
SERPINB10
SERPINB11
SERPINB12
SERPINB13
SERPINB2
SERPINB3
SERPINB4
SERPINB5
SERPINB6
SERPINB7
SERPINB8
SERPINB9
SERPINC1
SERPIND1
SERPINE1
SERPINE2
SERPINE3
SERPINF1
SERPINF2
SERPING1
SERPINH1
SERPINI1
SERPINI2
SLPI
SPAM1
ST14
SULF1
SULF2
TGM1
TGM2
TGM3
TGM4
TGM5
TGM6
TGM7
TIMP1
TIMP2
TIMP3
TIMP4
TLL1
TLL2
TMPRSS15
AMH
ANGPT1
ANGPT2
ANGPT4
ANGPTL1
ANGPTL2
ANGPTL3
ANGPTL4
ANGPTL5
ANGPTL6
ANGPTL7
AREG
ARTN
BDNF
BMP10
BMP15
BMP2
BMP3
BMP4
BMP5
BMP6
BMP7
BMP8A
BMP8B
BRINP2
BRINP3
BTC
C1QTNF9B
CBLN1
CBLN2
CBLN3
CBLN4
CCBE1
CCL1
CCL11
CCL13
CCL14
CCL15
CCL16
CCL17
CCL18
CCL19
CCL2
CCL20
CCL21
CCL22
CCL23
CCL24
CCL25
CCL26
CCL27
CCL28
CCL3
CCL3L3
CCL4
CCL4L1
CCL4L2
CCL5
CCL7
CCL8
CFC1
CFC1B
CHRD
CHRDL1
CHRDL2
CLCF1
CNTF
CRHBP
CRLF1
CRLF3
CRNN
CSF1
CSF2
CSF3
CSH1
CSH2
CSHL1
CTF1
CX3CL1
CXCL1
CXCL10
CXCL11
CXCL12
CXCL13
CXCL14
CXCL2
CXCL3
CXCL5
CXCL6
CXCL8
CXCL9
DHH
EBI3
EDA
EGF
EGFL6
EGFL7
EGFL8
EPGN
EPO
EREG
FASLG
FGF1
FGF10
FGF11
FGF12
FGF13
FGF14
FGF16
FGF17
FGF18
FGF19
FGF2
FGF20
FGF21
FGF22
FGF23
FGF3
FGF4
FGF5
FGF6
FGF7
FGF8
FGF9
FGFBP1
FGFBP2
FGFBP3
FIGF
FLG
FLG2
FLT3LG
FRZB
FST
FSTL1
FSTL3
GDF1
GDF10
GDF11
GDF15
GDF2
GDF3
GDF5
GDF6
GDF7
GDF9
GDNF
GH1
GH2
HBEGF
HCFC1
HCFC2
HGF
HGFAC
HHIP
HRNR
IFNA1
IFNA10
IFNA13
IFNA14
IFNA16
IFNA17
IFNA2
IFNA21
IFNA4
IFNA5
IFNA6
IFNA7
IFNA8
IFNB1
IFNE
IFNG
IFNK
IFNW1
IGF1
IGF2
IHH
IL10
IL11
IL12A
IL12B
IL13
IL15
IL16
IL17A
IL17B
IL17C
IL17D
IL17F
IL18
IL19
IL1A
IL1B
IL1F10
IL1F5
IL1F6
IL1F7
IL1F8
IL1F9
IL1RN
IL2
IL20
IL22
IL23A
IL24
IL25
IL26
IL3
IL34
IL4
IL5
IL6
IL7
IL9
INHA
INHBA
INHBB
INHBC
INHBE
INS
INS-IGF2
INSL3
INSL5
INSL6
ISM1
ISM2
KITLG
LEFTY1
LEFTY2
LEP
LIF
LTA
LTB
MDK
MEGF10
MEGF11
MEGF6
MEGF8
MEGF9
MST1
MST1L
MSTN
NGF
NODAL
NRG1
NRG2
NRG3
NRG4
NRTN
NTF3
NTF4
OSM
PDGFA
PDGFB
PDGFC
PDGFD
PF4
PF4V1
PGF
PIK3IP1
PPBP
PRL
PSPN
PTN
RPTN
S100A1
S100A10
S100A11
S100A12
S100A13
S100A14
S100A16
S100A2
S100A3
S100A4
S100A5
S100A6
S100A7
S100A7A
S100A7L2
S100A8
S100A9
S100B
S100G
S100P
S100Z
SCUBE1
SCUBE2
SCUBE3
SFRP1
SFRP2
SFRP4
SFRP5
SHH
TCHH
TCHHL1
TDGF1
TGFA
TGFB1
TGFB2
TGFB3
THPO
TNF
TNFSF10
TNFSF11
TNFSF12
TNFSF13
TNFSF13B
TNFSF14
TNFSF15
TNFSF18
TNFSF4
TNFSF8
TNFSF9
TPO
VEGFA
VEGFB
VEGFC
VWC2
VWC2L
WFIKKN1
WFIKKN2
WIF1
WNT1
WNT10A
WNT10B
WNT11
WNT16
WNT2
WNT2B
WNT3
WNT3A
WNT4
WNT5A
WNT5B
WNT6
WNT7A
WNT7B
WNT8A
WNT8B
WNT9A
WNT9B
XCL1
XCL2
ZFP91
ADAM1A
ADAM21P1
ADAM3A
ADAM3B
ADAM5
ADAM6
ANXA2P2
ANXA8L2
BPIFA4P
C17orf101
COL6A4P1
COL6A4P2
CST9L2
CTF2P
CTSL3P
CTSLP3
CTSLP6
CTSLP7
DMBT1P1
EGFEM1P
FGF7P2
gene_A6NLB4
HYALP1
KGFLP1
KGFLP2
LOC400696
LOC728715
LPAL2
MBL1P
MST1P2
NCRNA00083
NEPNP
NTF6B
OVOS1
OVOS2
PPBPP1
PRSS3P1
SERPINA13P
TNFSF12-TNFSF13
TNXA
UNQ5830
AXL
CD44
CHEK1
DACH1
DDR1
DKK1
DLL1
DLL4
DNMT1
ERBB2
GSK3B
HDAC1
ID1
IKBKB
ITGA2
ITGA4
ITGA6
ITGB1
JAG1
JAK2
KIT
KITLG
LATS1
LIN28A
LIN28B
MAML1
MERTK
MYC
MYCN
NANOG
NFKB1
NOS2
NOTCH2
PLAUR
POU5F1
PROM1
PTCH1
PTPRC
SAV1
SIRT1
SMO
SNAI1
SOX2
STAT3
TAZ
TGFBR1
THY1
TWIST1
TWIST2
WEE1
WNT1
WWC1
YAP1
ZEB1
ZEB2
ACTN1
ADGRE5
AKT1
ANKRD2
ANO1
ARHGAP24
ARHGEF2
ARHGEF40
ASIC1
ASIC2
ATOH1
BAG3
BCAR1
CAMKK2
CAPN2
CAV1
CD47
CD55
CDH1
CDH5
CIB2
CIB3
CITED2
CLRN1
CTNNA1
CXCR1
DTNA
EDN1
EGFR
EHD2
FLNA
FLT4
FSCN1
G3BP2
GNA11
GNAQ
GP1BA
GPSM2
HOTAIR
HOXA5
ILK
ITGA2B
ITGA5
ITGB3
JCAD
KCNK10
KCNK2
KCNK4
KCNMA1
KLF2
KLF3
MAPK1
MAPK14
MAPK3
MAPK8
MIR103A1
MIR146A
MIR181B1
MIR181B2
MIR195
MIR21
MRTFA
MYL2
MYO7A
OLR1
PANX1
PDLIM5
PDLIM7
PECAM1
PIEZO1
PIEZO1P1
PIEZO2
PKD2
PKHD1
PLPP3
POU4F3
PPFIA1
PPP1R12A
PRKAA1
PRKCA
PTGS2
PTK2
PXN
RAP2A
RELA
RHOA
ROCK1
RUNX2
SCNN1A
SELE
SMAD3
SOST
SRC
TAC1
TAOK3
TGFB1
TMC1
TMC2
TMEM150C
TNF
TRPA1
TRPC1
TRPC6
TRPM7
TRPV1
TRPV4
UNC50
VCL
VHL
WWTR1"""

genes = genes.split()

In [ ]:
gbm_from_excel=pd.read_csv("/Users/serkanturunc/Desktop/cbio research/EXCEL_500+DATA.csv")
gbm_from_excel.rename(columns={'Collection ID': 'PatientId'},inplace=True)
gbm_from_excel['WT YES']=(gbm_from_excel['IDH mutation status']=='WT').astype(int)
unique_pids=[]
indices_to_drop=[]
for index in gbm_from_excel.index:
    if gbm_from_excel.iloc[index]['PatientId'] in unique_pids:
        indices_to_drop.append(index)
    else:
        unique_pids.append(gbm_from_excel.iloc[index]['PatientId'])
indices_to_keep=[i for i in gbm_from_excel.index if i not in indices_to_drop]
gbm_from_excel_unique=gbm_from_excel.iloc[indices_to_keep]
for i in gbm_from_excel_unique['Sex'].index:
    if gbm_from_excel_unique['Sex'][i]=='Female':
        gbm_from_excel_unique['Sex'][i]=0
    elif gbm_from_excel_unique['Sex'][i]=='Male':
        gbm_from_excel_unique['Sex'][i]=1